# Shopping Cart Abandonment Prediction
## LSTM

LSTM model based on "Predicting purchasing intent: Automatic Feature Learning using
Recurrent Neural Networks" (Sheil, et.al.).

In [1]:
!wget https://github.com/mease/csci5502-cart-abandonment/raw/main/data/cart_abandon_data_seq_30.zip
!mkdir data
!unzip cart_abandon_data_seq_30.zip -d data

--2021-11-05 22:01:58--  https://github.com/mease/csci5502-cart-abandonment/raw/main/data/cart_abandon_data_seq_30.zip
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/mease/csci5502-cart-abandonment/main/data/cart_abandon_data_seq_30.zip [following]
--2021-11-05 22:01:58--  https://media.githubusercontent.com/media/mease/csci5502-cart-abandonment/main/data/cart_abandon_data_seq_30.zip
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160244411 (153M) [application/zip]
Saving to: ‘cart_abandon_data_seq_30.zip.1’

cart_abandon_data_s 100%[===================>] 152.82M   152MB/s    in

In [2]:
try:
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Embedding, Concatenate, LSTM, Dense

import numpy as np
import pickle

In [3]:
# Click sequence for all data points
sequence_length = 30

Load data:

In [4]:
with open('data/X_train.npy', 'rb') as f:
    X_train = np.load(f)
with open('data/y_train.npy', 'rb') as f:
    y_train = np.load(f)
with open('data/X_test.npy', 'rb') as f:
    X_test = np.load(f)
with open('data/y_test.npy', 'rb') as f:
    y_test = np.load(f)

In [5]:
for i in range(10):
    print(X_train[i], y_train[i])

[[ 1073  5595     1]
 [ 1073 34566     1]
 [ 1073 34566     1]
 [ 1073 10645     1]
 [ 1073 34566     1]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]] 0
[[ 4145 38491     6]
 [ 4145 46340     6]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0     0     0]
 [    0   

Determine vocabulary size for feature embeddings.

In [6]:
time_vocab_size = len(np.unique(np.concatenate((np.unique(X_train[:,:,0]),np.unique(X_test[:,:,0])))))
item_vocab_size = len(np.unique(np.concatenate((np.unique(X_train[:,:,1]),np.unique(X_test[:,:,1])))))
cat_vocab_size = len(np.unique(np.concatenate((np.unique(X_train[:,:,2]),np.unique(X_test[:,:,2])))))
print(f"Time vocab: {time_vocab_size}, Item vocab: {item_vocab_size}, Category vocab: {cat_vocab_size}")

Time vocab: 4369, Item vocab: 51758, Category vocab: 335


Define the model:

In [8]:
# Timestamp input and embedding
time_inp = Input(shape=(sequence_length,), name='time_input')
time_emb = Embedding(output_dim=10,
                     input_dim=time_vocab_size,
                     input_length=sequence_length,
                     embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.075, maxval=0.075),
                     mask_zero=True,
                     name='time_embedding')(time_inp)

# Item ID input and embedding
item_inp = Input(shape=(sequence_length,), name='item_input')
item_emb = Embedding(output_dim=100,
                     input_dim=item_vocab_size,
                     input_length=sequence_length,
                     embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.075, maxval=0.075),
                     mask_zero=True,
                     name='item_embedding')(item_inp)

# Category input and embedding
cat_inp = Input(shape=(sequence_length,), name='category_input')
cat_emb = Embedding(output_dim=10,
                    input_dim=cat_vocab_size,
                    input_length=sequence_length,
                     embeddings_initializer=tf.keras.initializers.RandomUniform(minval=-0.075, maxval=0.075),
                    mask_zero=True,
                    name='category_embedding')(cat_inp)

# Concatenate embeddings
concat_emb = Concatenate(axis=2, name='concat_embeddings')([time_emb, item_emb, cat_emb])

# LSTM
lstm1 = LSTM(256, return_sequences=True, name='lstm1')(concat_emb)
lstm2 = LSTM(256, return_sequences=True, name='lstm2')(lstm1)
lstm3 = LSTM(256, name='lstm3')(lstm2)

# Output layer
out = Dense(1, activation='sigmoid', name='output')(lstm3)

model = Model(inputs=[time_inp, item_inp, cat_inp], outputs=[out])
model.compile(loss='binary_crossentropy',
              optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
              metrics=[tf.keras.metrics.AUC()])

print(model.summary())

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
time_input (InputLayer)         [(None, 30)]         0                                            
__________________________________________________________________________________________________
item_input (InputLayer)         [(None, 30)]         0                                            
__________________________________________________________________________________________________
category_input (InputLayer)     [(None, 30)]         0                                            
__________________________________________________________________________________________________
time_embedding (Embedding)      (None, 30, 10)       43690       time_input[0][0]                 
______________________________________________________________________________________________

Train model:

In [9]:
model.fit([X_train[:,:,0], X_train[:,:,1], X_train[:,:,2]], y_train,
          validation_data=([X_test[:,:,0], X_test[:,:,1], X_test[:,:,2]], y_test),
          epochs=4,
          batch_size=256)

Epoch 1/4
28836/28836 [==============================] - 1809s 62ms/step - loss: 0.1761 - auc: 0.8049 - val_loss: 0.1705 - val_auc: 0.8254
Epoch 2/4
28836/28836 [==============================] - 1850s 64ms/step - loss: 0.1671 - auc: 0.8362 - val_loss: 0.1690 - val_auc: 0.8299
Epoch 3/4
28836/28836 [==============================] - 1815s 63ms/step - loss: 0.1634 - auc: 0.8470 - val_loss: 0.1691 - val_auc: 0.8304
Epoch 4/4
28836/28836 [==============================] - 1761s 61ms/step - loss: 0.1602 - auc: 0.8551 - val_loss: 0.1702 - val_auc: 0.8302
